In [5]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
df = pd.read_csv("../data/li_chess_for_exp.csv", index_col=0) # relative path
df["explanation"] = ""
df.head()

,moves,explanation
0,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,
1,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,
2,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,
3,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,
4,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,


In [18]:
df.shape

(5000, 2)

In [14]:
load_dotenv()
openai_api_key = os.getenv("openai_api_key")

In [15]:
client = OpenAI(api_key=openai_api_key)

In [6]:
def gen_cot_prompt(prev_moves : str) -> str:
    prev_moves = prev_moves.split(" ")
    cot_moves = [f"White: {prev_moves[i]}, Black: {prev_moves[i+1]}" if i + 1 < len(prev_moves) else f"White: {prev_moves[i]}" for i in range(0, len(prev_moves), 2)]
    return "\n".join(cot_moves)

In [11]:
# testing
print(len(gen_cot_prompt(df.iloc[2]["moves"]).split(" ")))
print(gen_cot_prompt(df.iloc[2]["moves"]))

92
White: e4, Black: e5
White: d3, Black: d6
White: Be3, Black: c6
White: Be2, Black: b5
White: Nd2, Black: a5
White: a4, Black: c5
White: axb5, Black: Nc6
White: bxc6, Black: Ra6
White: Nc4, Black: a4
White: c3, Black: a3
White: Nxa3, Black: Rxa3
White: Rxa3, Black: c4
White: dxc4, Black: d5
White: cxd5, Black: Qxd5
White: exd5, Black: Be6
White: Ra8+, Black: Ke7
White: Bc5+, Black: Kf6
White: Bxf8, Black: Kg6
White: Bxg7, Black: Kxg7
White: dxe6, Black: Kh6
White: exf7, Black: Nf6
White: Rxh8, Black: Nh5
White: Bxh5, Black: Kg5
White: Rxh7, Black: Kf5
White: Qf3+, Black: Ke6
White: Bg4+, Black: Kd6
White: Rh6+, Black: Kc5
White: Qe3+, Black: Kb5
White: c4+, Black: Kb4
White: Qc3+, Black: Ka4
White: Bd1#


In [16]:
system_content_cot = '''
    Act as a chess master.
    You will be provided with a list of chess move pairs in Algebraic Notation where 1st move is by White and 2nd by Black.
    Generate a concise and short rationale behind each pair of moves for all the pairs provided.
    After the pair based analysis, you have to write a short summary on why the last move was taken, who is likely to be the winner and what strategy did the player use throughout the game.
    Use the heading for your answer: Game analysis, winner and summary of strategy.
    '''

In [17]:
oai_model = "gpt-3.5-turbo-0125"
# oai_model = "gpt-4-turbo"
# oai_model = "gpt-4o"

In [28]:
def generate_explanation(client: OpenAI, model_name: str, df: pd.DataFrame, system_content: str, save_path: str, save_interval: int = 20) -> pd.DataFrame:
    return_df = df.copy()

    for i in range(df.shape[0]):
        print(f"[Info] Generating explanation for row {i}")
        moves = df.iloc[i]["moves"]

        user_prompt_move = gen_cot_prompt(moves)
        user_content_cot = f"Move pairs are - \n{user_prompt_move}"
        
        stream = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_content_cot}
            ],
            stream=True,
        )
        
        explanation = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                explanation += chunk.choices[0].delta.content
        return_df.loc[i, "explanation"] = explanation.strip()
        
        # saving file at each interval
        if (i + 1) % save_interval == 0:
            header = (i == 0)
            return_df.to_csv(save_path, mode="w", header=header, index=False)
            print(f"[Info] Saved progress as csv in {save_path} at iteration {i + 1}")

    # Save the final DataFrame after the loop
    print("[Info] Final data saved")
    return return_df


In [29]:
# for testing
temp = df.iloc[:20]

In [30]:
target_df = generate_explanation(OpenAI(api_key = openai_api_key), oai_model, temp, system_content_cot, "./interval_data/interval.csv", 5)
target_df.to_csv(f"./exp_data/lichess_exp.csv")
print("[Info] Final CSV generated")

[Info] Generating explanation for row 0
[Info] Generating explanation for row 1
[Info] Generating explanation for row 2
[Info] Generating explanation for row 3
[Info] Generating explanation for row 4
[Info] Saved progress as csv in ./interval_data/interval.csv at iteration 5
[Info] Generating explanation for row 5
[Info] Generating explanation for row 6
[Info] Generating explanation for row 7
[Info] Generating explanation for row 8
[Info] Generating explanation for row 9
[Info] Saved progress as csv in ./interval_data/interval.csv at iteration 10
[Info] Generating explanation for row 10
[Info] Generating explanation for row 11
[Info] Generating explanation for row 12
[Info] Generating explanation for row 13
[Info] Generating explanation for row 14
[Info] Saved progress as csv in ./interval_data/interval.csv at iteration 15
[Info] Generating explanation for row 15
[Info] Generating explanation for row 16
[Info] Generating explanation for row 17
[Info] Generating explanation for row 18
[

OSError: Cannot save file into a non-existent directory: 'exp_data'